## 2.10 外部プログラムの呼び出し
Juliaで外部プログラムを使用するには，バッククオートを用いる．  
ただし，バッククオートでくくったコマンドを，run(), read(), open()などで利用した時に実行される．  
また，Windowsではコマンドをコマンドプロンプトで叩くわけではなく，「ファイル名を指定して実行」で実行する場合に相当する．そのため，ただコマンドを入れただけでは`no such file or directory`というエラーになってしまう．

In [1]:
run(`echo hello`)

LoadError: IOError: could not spawn `echo hello`: no such file or directory (ENOENT)

そこで，`cmd /c ～`とすればコマンドラインを叩ける．

In [2]:
run(`cmd /c echo hello`)

hello


Process(`cmd /c echo hello`, ProcessExited(0))

もちろんPathが通ってるコマンドや，絶対パスを指定すればそのままでも実行できる．

In [4]:
run(`C:\\Python38\\python hello.py`)

hello python from julia


Process(`'C:\Python38\python' hello.py`, ProcessExited(0))

## 3.2 ファイル入出力
### 3.2.1 ファイルとストリーム
低レベルIOは以下のように行う．

In [12]:
lines = ["file", "I/O", "test"]
# ファイルの書き込み．"w"->"a"だと追加書き込み
open("test.txt", "w") do f
    for line in lines
        println(f, line)
    end
end

In [13]:
# 書き込んだファイルの読み込み
open("test.txt") do f
    for line in eachline(f)
        println(line)
    end
end

file
I/O
test


### 3.2.4 JSONファイルの入出力
`using Pkg` `Pkg.add("JSON")` しておく

In [20]:
using JSON

In [27]:
# JSON形式データの作成
data = Dict(
    "構造"=>"鉄筋コンクリート造", 
    "階床"=>"8階建て",
    "用途"=>["事務所","店舗"]
)
jsondata = JSON.json(data)
open("building.json", "w") do f
    print(f, jsondata)
end

In [29]:
# JSON形式データの解析
println(JSON.parse(jsondata))
println(JSON.parsefile("building.json"))

Dict{String,Any}("用途" => Any["事務所", "店舗"],"階床" => "8階建て","構造" => "鉄筋コンクリート造")
Dict{String,Any}("用途" => Any["事務所", "店舗"],"階床" => "8階建て","構造" => "鉄筋コンクリート造")


### TOMLファイル入出力
TOMLは設定ファイルとして有力なフォーマット．Julia関係ライブラリの依存関係もProject.tomlで記述される．  
公式のTOML.jlがあるので，インストールする．`Pkg.add("TOML")`  

In [31]:
using TOML

試しにTOML.jlのProject.tomlをパースしてみる．
`Pkg.add("HTTP")`

In [35]:
using HTTP

In [43]:
res = HTTP.request("GET", "https://raw.githubusercontent.com/JuliaLang/TOML.jl/master/Project.toml")
tomldata = String(res.body)
print(tomldata)

name = "TOML"
uuid = "fa267f1f-6049-4f14-aa54-33bafae1ed76"
version = "1.0.1"

[deps]
Dates = "ade2ca70-3891-5945-98fb-dc099432e06a"

[compat]
julia = "1"

[extras]
Test = "8dfed614-e22c-5e08-85e1-65c5234f0b40"

[targets]
test = ["Test"]


In [72]:
# TOMLデータの解析
data = TOML.parse(tomldata)

Dict{String,Any} with 7 entries:
  "deps"    => Dict{String,Any}("Dates"=>"ade2ca70-3891-5945-98fb-dc099432e06a")
  "name"    => "TOML"
  "extras"  => Dict{String,Any}("Test"=>"8dfed614-e22c-5e08-85e1-65c5234f0b40")
  "uuid"    => "fa267f1f-6049-4f14-aa54-33bafae1ed76"
  "compat"  => Dict{String,Any}("julia"=>"1")
  "targets" => Dict{String,Any}("test"=>["Test"])
  "version" => "1.0.1"

In [74]:
# TOMLデータの変更と保存
data["targets"]["test"] = ["Test2"]
open("project.toml", "w") do f
    TOML.print(f, data)
end

### CSVファイルの入出力
`Pkg.add("CSV")` `Pkg.add("DataFrames")`しておく

In [69]:
using CSV
using DataFrames

In [75]:
# データの作成
data = rand(100, 4)
df = DataFrame(data)
# データの書き込み．
CSV.write("data.csv", df)

"data.csv"

In [92]:
# データの読み込み
df2 = DataFrame(CSV.File("data.csv"))
sum(Matrix(df2)-Matrix(df)) # 書き込んだデータと誤差がないことを確認

0.0